# Python deltalake 0.9.0: Using Optimize and Vacuum in append-only Delta Lake workloads

Delta Lake tables are an excellent way to store large datasets. With the Python package, you can write ETL jobs that write small amounts of data at a time, when a full distributed cluster might be overkill. This can be useful for workloads that require periodically pulling in data to add to the data lake. However, writing small amounts of data at a time can cause there to be a huge number of files in the table, which can slow down queries.

In deltalake 0.9.0, we added the `optimize` method to the `DeltaTable` class, which performs file compaction. Running this periodically on a table will reduce the number of files in the table, which will speed up queries.

This is very helpful for workloads that append frequently. For example, if you have a table that is appended to every 10 minutes, after a year you will have 52,560 files in the table. If the table is partitioned by another dimension, you will have 52,560 files per partition; with just 100 unique values that's millions of files. By running `optimize` periodically, you can reduce the number of files in the table to a more manageable number.

Typically, you will run optimize less frequently than you append data. If possible, you might run optimize once you know you have finished writing to a particular partition. For example, on a table partitioned by date, you might append data every 10 minutes, but only run optimize once a day at the end of the day. This will ensure you don't need to compact the same data twice.

We'll look at a simple example of this type of workload, and how `optimize` and `vacuum` can be used to improve performance.

In [1]:
import pyarrow as pa
import pyarrow.compute as pc
from deltalake import DeltaTable, write_deltalake
from datetime import datetime, timedelta
import itertools

To simulate a workload that pulls in new data periodically, we wrote a function that generates a new set of random data given a timestamp. We'll pass a sequence of hours to this, but the frequency could be anything.

In [2]:
def record_observations(date: datetime) -> pa.Table:
    """Pulls data for a certain datetime"""
    nrows = 1000
    return pa.table({
        "date": pa.array([date.date()] * nrows),
        "timestamp": pa.array([date] * nrows),
        "value": pc.random(nrows),
    })

# Example of output
record_observations(datetime(2021, 1, 1, 12)).to_pandas()

,date,timestamp,value
0,2021-01-01,2021-01-01 12:00:00,0.982481
1,2021-01-01,2021-01-01 12:00:00,0.212515
2,2021-01-01,2021-01-01 12:00:00,0.239757
3,2021-01-01,2021-01-01 12:00:00,0.078596
4,2021-01-01,2021-01-01 12:00:00,0.564797
...,...,...,...
995,2021-01-01,2021-01-01 12:00:00,0.724803
996,2021-01-01,2021-01-01 12:00:00,0.235261
997,2021-01-01,2021-01-01 12:00:00,0.205549
998,2021-01-01,2021-01-01 12:00:00,0.337829


First, we'll write 100 hours worth of data to the table.

In [3]:
# Every hour starting at midnight on 2021-01-01
hours_iter = (datetime(2021, 1, 1) + timedelta(hours=i) for i in itertools.count())

# Write 100 hours worth of data
for timestamp in itertools.islice(hours_iter, 100):
    write_deltalake(
        "observation_data",
        record_observations(timestamp),
        partition_by=["date"],
        mode="append",
    )

We can now load out table's state with `DeltaTable("path/to/table")`. How do we tell how many files there are? We can use the `.files()` method to get the list of files in the current version of the table.

In [4]:
dt = DeltaTable("observation_data")
# We now have 100 files in our table
len(dt.files())

100

We have 100 files, but how many partitions do we have? The `get_add_actions()` method gives us statistics for every file. With a little data wrangling, we can get the unique values for the `date` partition column.

In [5]:
# But there are only 5 unique partitions
dt.get_add_actions(flatten=True).column("partition.date").unique().sort()

[
  2021-01-01,
  2021-01-02,
  2021-01-03,
  2021-01-04,
  2021-01-05
]

Now we can run `optimize()` on our table. This compacts the 100 files into a single file per partition. Since we have 5 partitions, it adds 5 files to the table. The previous 100 files are removed from the table. All this is show in the metrics output by `optimize()`.

In [6]:
dt.optimize()

{'numFilesAdded': 5,
 'numFilesRemoved': 100,
 'filesAdded': {'min': 39000,
  'max': 238282,
  'avg': 198425.6,
  'totalFiles': 5,
  'totalSize': 992128},
 'filesRemoved': {'min': 10244,
  'max': 10244,
  'avg': 10244.0,
  'totalFiles': 100,
  'totalSize': 1024400},
 'partitionsOptimized': 5,
 'numBatches': 1,
 'totalConsideredFiles': 100,
 'totalFilesSkipped': 0,
 'preserveInsertionOrder': True}

Now when we check the number of files, we see that we have 5 files, one per partition.

In [7]:
# After running optimize, we have an equal number of files as partitions
len(dt.files())

5

## Handling incremental updates

Above, we optimized a table when the entire table had too many files. But when we incrementally update the table, we'll only have extra files in the new partitions. Let's take a look at how we handle incremental updates.

We'll add another 24 hours worth of data.

In [8]:
# Add another 24 hours of data
for timestamp in itertools.islice(hours_iter, 24):
    write_deltalake(
        dt,
        record_observations(timestamp),
        partition_by=["date"],
        mode="append",
    )

Now we can use `get_add_actions()` again to introspect the table state. We can see that `2021-01-06` has only a few hours of data so far, so we don't want to optimize that yet. But `2021-01-05` has all 24 hours of data, so it's ready to be optimized.

In [9]:
dt.get_add_actions(flatten=True).to_pandas()["partition.date"].value_counts().sort_index()

2021-01-01     1
2021-01-02     1
2021-01-03     1
2021-01-04     1
2021-01-05    21
2021-01-06     3
Name: partition.date, dtype: int64

To optimize a single partition, you can pass in a `partition_filters` argument speficying which partitions to optimize.

In [10]:
dt.optimize(partition_filters=[("date", "=", "2021-01-05")])

{'numFilesAdded': 1,
 'numFilesRemoved': 21,
 'filesAdded': {'min': 238282,
  'max': 238282,
  'avg': 238282.0,
  'totalFiles': 1,
  'totalSize': 238282},
 'filesRemoved': {'min': 10244,
  'max': 39000,
  'avg': 11613.333333333334,
  'totalFiles': 21,
  'totalSize': 243880},
 'partitionsOptimized': 1,
 'numBatches': 1,
 'totalConsideredFiles': 21,
 'totalFilesSkipped': 0,
 'preserveInsertionOrder': True}

## Vacuuming after optimizing

When we optimize a table, we remove the old files from the table. However, these files are still in the table's transaction log. This is useful for tables where we might delete data, since we can look at old versions of the table. But for tables where we only append data and optimize partitions, the old files are mostly redundant.

To remove them, we can use the `vacuum()` method. By default, this will remove all files that are older than 7 days. You can pass in a `retention_hours` argument to change this. However, for safety this argument won't allow windows that are too recent, unless you also pass the `enforce_retention_period=False` argument. Since for our workload the old files are redundant, we are okay with `rentention_hours=0`.

In [11]:
dt.vacuum(retention_hours=0, enforce_retention_duration=False, dry_run=False)

['date=2021-01-02/42-24b42777-c22b-4cdd-8847-0c04f2cb6ac6-0.parquet',
 'date=2021-01-02/39-29b0f345-fe21-4acf-becd-bd2cc9108718-0.parquet',
 'date=2021-01-02/31-a2ea6e9c-fe3d-4676-8da0-0b14c9c99c74-0.parquet',
 'date=2021-01-02/46-a730fcca-fd9c-4d86-b9f7-914f60185af3-0.parquet',
 'date=2021-01-02/36-23e730c3-14ab-4101-a543-e5239652678e-0.parquet',
 'date=2021-01-02/40-ee6a6111-facd-4d29-9973-35426997bce4-0.parquet',
 'date=2021-01-02/32-36695bca-52e9-4872-bf54-4e0780e6c103-0.parquet',
 'date=2021-01-02/27-05cdba48-93e3-4c4e-92ec-3b3c96db052d-0.parquet',
 'date=2021-01-02/33-4edcbe0a-b43b-4bce-a20c-4141ae911f0b-0.parquet',
 'date=2021-01-02/37-5d6c2c19-5261-4bbf-9b1a-02f1c5e723a9-0.parquet',
 'date=2021-01-02/25-d613b9ee-f3fa-4136-b1b8-0151fdec2c19-0.parquet',
 'date=2021-01-02/34-c975537c-2a1d-42cc-98cf-c5754a6c1c3e-0.parquet',
 'date=2021-01-02/41-3ac72047-9a87-406c-8b1d-4de650b7987a-0.parquet',
 'date=2021-01-02/35-d274fede-d209-4990-9375-9b49a341a912-0.parquet',
 'date=2021-01-02/44